In [92]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
import calendar

In [3]:
data = pd.read_csv('initial_dataset.csv')

In [4]:
data_cities = data[~(data['ville'] == 'Paris')]

In [5]:
data_cities.loc[:, 'processed_date'] = data_cities.loc[:, 'date'].apply(lambda row : datetime(int(row.split('-', 2)[0]), int(row.split('-', 2)[1]), int(row.split('-', 2)[2])))

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [6]:
data_cities

,date,equipement,ville,CA,processed_date
3285,2017-01-01,ordinateur,Bordeaux,270.941633,2017-01-01
3286,2017-01-02,ordinateur,Bordeaux,1894.634278,2017-01-02
3287,2017-01-03,ordinateur,Bordeaux,1963.790266,2017-01-03
3288,2017-01-04,ordinateur,Bordeaux,1560.772008,2017-01-04
3289,2017-01-05,ordinateur,Bordeaux,1370.575521,2017-01-05
...,...,...,...,...,...
9850,2019-12-27,accessoires,Mont de Marsan,63.075136,2019-12-27
9851,2019-12-28,accessoires,Mont de Marsan,18.267467,2019-12-28
9852,2019-12-29,accessoires,Mont de Marsan,94.673079,2019-12-29
9853,2019-12-30,accessoires,Mont de Marsan,12.928690,2019-12-30


In [7]:
data_cities['previous_year'] = data_cities['processed_date'].apply(lambda row : row - timedelta(days=365))
data_cities['previous_year']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


3285   2016-01-02
3286   2016-01-03
3287   2016-01-04
3288   2016-01-05
3289   2016-01-06
          ...    
9850   2018-12-27
9851   2018-12-28
9852   2018-12-29
9853   2018-12-30
9854   2018-12-31
Name: previous_year, Length: 6570, dtype: datetime64[ns]

In [11]:
data_cities

,date,equipement,ville,CA,processed_date,previous_year
3285,2017-01-01,ordinateur,Bordeaux,270.941633,2017-01-01,2016-01-02
3286,2017-01-02,ordinateur,Bordeaux,1894.634278,2017-01-02,2016-01-03
3287,2017-01-03,ordinateur,Bordeaux,1963.790266,2017-01-03,2016-01-04
3288,2017-01-04,ordinateur,Bordeaux,1560.772008,2017-01-04,2016-01-05
3289,2017-01-05,ordinateur,Bordeaux,1370.575521,2017-01-05,2016-01-06
...,...,...,...,...,...,...
9850,2019-12-27,accessoires,Mont de Marsan,63.075136,2019-12-27,2018-12-27
9851,2019-12-28,accessoires,Mont de Marsan,18.267467,2019-12-28,2018-12-28
9852,2019-12-29,accessoires,Mont de Marsan,94.673079,2019-12-29,2018-12-29
9853,2019-12-30,accessoires,Mont de Marsan,12.928690,2019-12-30,2018-12-30


In [12]:
CA_cum = data_cities.groupby(by=['ville', 'equipement']).sum().reset_index()
CA_cum

,ville,equipement,CA
0,Bordeaux,accessoires,7.785279e+04
1,Bordeaux,ordinateur,3.217173e+06
2,Bordeaux,téléphone,6.566389e+05
3,Mont de Marsan,accessoires,7.569744e+04
4,Mont de Marsan,ordinateur,3.326700e+06
5,Mont de Marsan,téléphone,6.833783e+05


In [142]:
def CA_cum_f(row) :
    ca_cum = CA_cum[(CA_cum['ville'] == row['ville'].iloc[0]) & \
                  (CA_cum['equipement'] == row['equipement'].iloc[0])]['CA']
    return ca_cum.iloc[0]


data_cities = data_cities.assign( CA_cumul = lambda row : CA_cum_f(row) )

In [144]:
def CA_previous_year(row) :
    ca = data_cities[(data_cities['processed_date'] == row['previous_year'].iloc[0]) & \
                       (data_cities['ville'] == row['ville'].iloc[0]) & \
                       (data_cities['equipement'] == row['equipement'].iloc[0])]['CA']
    print(type(ca))
    return ca.iloc[0]


data_cities = data_cities.assign( CA_previous_year = lambda row : CA_previous_year(row) )

<class 'pandas.core.series.Series'>


IndexError: single positional indexer is out-of-bounds

In [93]:
data_cities['weekday'] = data_cities['processed_date'].apply(lambda row : calendar.day_name[row.weekday()] )

In [95]:
data_cities['is_weekend'] = data_cities['weekday'].apply(lambda row : row in ['Sunday', 'Saturday'] )

In [107]:
(~data_cities.isna()).sum()

date                6570
equipement          6570
ville               6570
CA                  6570
processed_date      6570
previous_year       6570
CA_previous_year       0
weekday             6570
is_weekend          6570
CA_cumul               0
dtype: int64

In [143]:
data_cities

,date,equipement,ville,CA,processed_date,previous_year,CA_previous_year,weekday,is_weekend,CA_cumul
3285,2017-01-01,ordinateur,Bordeaux,270.941633,2017-01-01,2016-01-02,None,Sunday,True,3.217173e+06
3286,2017-01-02,ordinateur,Bordeaux,1894.634278,2017-01-02,2016-01-03,None,Monday,False,3.217173e+06
3287,2017-01-03,ordinateur,Bordeaux,1963.790266,2017-01-03,2016-01-04,None,Tuesday,False,3.217173e+06
3288,2017-01-04,ordinateur,Bordeaux,1560.772008,2017-01-04,2016-01-05,None,Wednesday,False,3.217173e+06
3289,2017-01-05,ordinateur,Bordeaux,1370.575521,2017-01-05,2016-01-06,None,Thursday,False,3.217173e+06
...,...,...,...,...,...,...,...,...,...,...
9850,2019-12-27,accessoires,Mont de Marsan,63.075136,2019-12-27,2018-12-27,None,Friday,False,3.217173e+06
9851,2019-12-28,accessoires,Mont de Marsan,18.267467,2019-12-28,2018-12-28,None,Saturday,True,3.217173e+06
9852,2019-12-29,accessoires,Mont de Marsan,94.673079,2019-12-29,2018-12-29,None,Sunday,True,3.217173e+06
9853,2019-12-30,accessoires,Mont de Marsan,12.928690,2019-12-30,2018-12-30,None,Monday,False,3.217173e+06


In [11]:
class Person :
    
    def __init__(self, name, age) :
        self.name = name
        self.age = age
        
    def talk(self) :
        return "My name is {} and I am {} years old".format(self.name, self.age)
    
    def __add__(self, p) :
        return Person(self.name + p.name, self.age + p.age)
    

In [12]:
p1 = Person('alex', 40)
p2 = Person('seb', 30)

In [13]:
p = p1 + p2

In [14]:
p.talk()

'My name is alexseb and I am 70 years old'

In [175]:
from datetime import datetime

        
def file_name_arg(name_file) :
    def write_to_file(func) :
        def wrapper(*args, **kwargs) :
            d = func(*args, **kwargs)
            print(d)
            with open(name_file, 'a') as f :
                f.write(str(d) + '\n')
        return wrapper
    return write_to_file


    
class MyDatetime :
    
    birthday = datetime(2019, 11, 23)
        
    @file_name_arg('date_time_file2.txt')
    def diff(self, dt) :
        return dt - self.__class__.birthday
        
    
    @file_name_arg('date_time_file2.txt')
    def __str__(self) :
        return self.__class__.birthday.timestamp()
    
    @property
    def age_today(self) :
        return self.diff(datetime.today())
        

In [178]:
mbd = MyDatetime()
mbd.diff(datetime(2020,4,21))
mbd.age_today

150 days, 0:00:00
320 days, 17:25:03.239674


In [167]:
str(mbd.diff(datetime.today()))

320 days, 17:18:39.234425


'None'